In [1]:
import cycledata as cd
import pandas as pd
import numpy as np
import seaborn
%matplotlib inline

In [ ]:
# Import seperated weeks for all years
WD = {}
WE = {}
WD, WE = cd.Import('split')

In [ ]:
# Import daily counts for all years
WDdays = {}
WEdays = {}
WDdays, WEdays = cd.Import('daily')

In [ ]:
# Import Multi-Year Daily Usage
WDs, WEs = Import('multi')

In [2]:
# Import Full Years (non-separated weeks)
FullYear = {}
FullYear = cd.Import('fullyear')

Reading: 2012FullYear.csv
Reading: 2013FullYear.csv
Reading: 2014FullYear.csv
Reading: 2015FullYear.csv
Reading: 2016FullYear.csv


In [3]:
year15 = FullYear['2015'].copy()
year15.sort_values('s_date', inplace=True)
year15.set_index(year15.s_date, drop=True, inplace=True)
DF = year15[:'18-12-2015'].copy()
# Find nearest Sunday
j = 1
while(int(DF[-j:(len(DF) - j + 1)].index.dayofweek) != 6):
    j += 1
DF = DF[:(-j+1)]
# 4 week section of 2015 to be used for modelling an validation
recent = DF.last('4W')
# 10 weekends section of 2015 to be used for modelling and validation
recentWE = DF.last('5W')

In [4]:
output, absent, errors = cd.RunWEModel(recentWE)

In [ ]:
# Create Seasonal Indexes
WDseasons, WEseasons = cd.GetSeasonalIndex()

In [ ]:
# Find most recent 2 weeks
recent = cd.RecentFull(FullYear)
recent.info()

In [ ]:
# Find most recent 2 weeks of data to Model
recentWD, recentWE = cd.Recent(WD, WE)
recentWD.info()
recentWE.info()

In [ ]:
# Model subset of data for individual station
FullModel = cd.Model(recent, 14)
FullModel.PreProcess(separate=True)
FullModel.WD.head()

In [ ]:
# Adjust any overlaps within the year frames
FY = {}
WD = {}
WE = {}
FY, WD, WE = cd.AdjustOverlap(FullYear)

In [ ]:
# Create daily average of yearframes
WDdaily, WEdaily = cd.YearsDaily(WD, WE)

In [ ]:
# Fetch AddressBook and stations missing from addressbook
addressbook, missingStations = cd.GetAddressBook()

In [ ]:
# Read AddressBook 
addressbook = pd.read_csv("Bike_Stations.csv")

In [ ]:
# Create Distances matrix (WARNING: long runtime ~30mins)
distances = cd.GetDistances(addressbook)

In [ ]:
# Read Distances matrix
distances = pd.read_csv("Bike_Station_Distances.csv", index_col=0)

In [ ]:
# Create test dataframe of a single day from recent (may need to adjust date)
testday = recent['28-2-2016'].copy().reset_index(drop=True)

In [ ]:
# Create adjacency matrix of testday 08:00 - 09:00
adjacency, am_peak = cd.Transform(testday, 32, 36)

In [ ]:
# Calculate average speeds for testday (WARNING: can be long runtime)
updated = cd.CalcSpeeds(testday)

In [ ]:
# Run SARIMA model of Weekdays using most recent dataframe
output, absent, errors = cd.RunWDModel(recent)

In [ ]:
# Run Weekend models using most recent dataframe
output, absent, errors = cd.RunWEModel(recent)